In [1]:
from predibase import Predibase, FinetuningConfig, DeploymentConfig

pb = Predibase(api_token="각자 API입력")
# https://app.predibase.com/

Connected to Predibase as User(id=b5555c02-4e64-48dd-b993-2f03a91a605d, username=sehoonpark@hanyang.ac.kr)

In [3]:
dataset = pb.datasets.from_file("./dataset/dataset.jsonl", name="medicine_dataset")

In [4]:
# Create an adapter repository
repo = pb.repos.create(name="medicine_suggest-model", description="medicine suggetion chatbot", exists_ok=True)

# Start a fine-tuning job, blocks until training is finished
adapter = pb.adapters.create(
    config=FinetuningConfig(
        base_model="solar-1-mini-chat-240612"
    ),
    dataset=dataset, # Also accepts the dataset name as a string
    repo=repo,
    description="initial model with defaults"
)

Successfully requested finetuning of solar-1-mini-chat-240612 as `medicine_suggest-model/1`. (Job UUID: d7b36cb4-ee59-4150-aec8-88965bc41e6c).

Watching progress of finetuning job d7b36cb4-ee59-4150-aec8-88965bc41e6c. This call will block until the job has finished. Canceling or terminating this call will NOT cancel or terminate the job itself.

Job is starting. Total queue time: 0:00:46         
Waiting to receive training metrics...

┌────────────┬────────────┬─────────────────┐
│ checkpoint │ train_loss │ validation_loss │
├────────────┼────────────┼─────────────────┤
│     1      │   0.9765   │        --       │
│     2      │   1.0895   │        --       │
│     3      │   0.6226   │        --       │
└────────────┴────────────┴─────────────────┘


In [5]:
# Get adapter, blocking call if training is still in progress
adapter = pb.adapters.get("medicine_suggest-model/1")
adapter

Adapter(repo='medicine_suggest-model', tag=1, archived=False, base_model='solar-1-mini-chat-240612', description='initial model with defaults', artifact_path='d7b36cb4-ee59-4150-aec8-88965bc41e6c/2c32e461fb794286b36b69a9f21b13f2/artifacts/model/model_weights', finetuning_error=None, finetuning_job_uuid='d7b36cb4-ee59-4150-aec8-88965bc41e6c')

In [6]:
# Specify the deployment by name
lorax_client = pb.deployments.client("medicine_suggest-model")
print(lorax_client.generate("""<<SYS>>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<</SYS>>

[INST] 벤포벨에스정의 효능은 무엇입니까? [/INST]""", max_new_tokens=100).generated_text)

NotFoundError: 